In [1]:
import numpy as np
import pandas as pd
import os
import math
import random

In [2]:
class FuzzyMMC:
    def __init__(self, sensitivity=1, exp_bound=1):
        # konstruktor
        self.sensitivity = sensitivity
        self.hyperboxes = None
        self.classes = np.array([])
        self.exp_bound = exp_bound
        
    def membership(self, pattern):
        # racuna pripadnost i vraca niz pripadnosti svakom hiperbiksu
        min_pts = self.hyperboxes[:, 0, :]
        max_pts = self.hyperboxes[:, 1, :]

        a = np.maximum(0, (1 - np.maximum(0, (self.sensitivity * np.minimum(1, pattern - max_pts)))))
        b = np.maximum(0, (1 - np.maximum(0, (self.sensitivity * np.minimum(1, min_pts - pattern)))))
        return np.sum(a + b, axis=1) / (2 * len(pattern))
    
    def overlap_contract(self, index):
        #proveravamo da li se hiperboksevi preklapaju
        contracted = False
        for test_box in range(len(self.hyperboxes)):
            if self.classes[test_box] == self.classes[index]:
                # ignorisemo preklapanje hiperbokseva iste klase
                continue
            expanded_box = self.hyperboxes[index]
            box = self.hyperboxes[test_box]
            
            vj, wj = expanded_box #onaj za koji gledamo da li se preklapa sa nekim
            vk, wk = box

            # moguci slucajevi preklapanja
            # trazimo najmanje preklapanje
            delta_new = delta_old = 1
            min_overlap_index = -1
            for i in range(len(vj)):
                if vj[i] < vk[i] < wj[i] < wk[i]:
                    delta_new = min(delta_old, wj[i] - vk[i])
                elif vk[i] < vj[i] < wk[i] < wj[i]:
                    delta_new = min(delta_old, wk[i] - vj[i])
                
                elif vj[i] < vk[i] < wk[i] < wj[i]:
                    delta_new = min(delta_old, min(wj[i] - vk[i], wk[i] - vj[i]))

                elif vk[i] < vj[i] < wj[i] < wk[i]:
                    delta_new = min(delta_old, min(wj[i] - vk[i], wk[i] - vj[i]))

                if delta_old - delta_new > 0:
                    min_overlap_index = i
                    delta_old = delta_new

            # ako ima preklapanja, 
            # gledamo po kojoj strani smanjujemo hiperboks
            if min_overlap_index >= 0:
                i = min_overlap_index
                # We need to contract the expanded box
                if vj[i] < vk[i] < wj[i] < wk[i]:
                    vk[i] = wj[i] = (vk[i] + wj[i])/2

                elif vk[i] < vj[i] < wk[i] < wj[i]:
                    vj[i] = wk[i] = (vj[i] + wk[i])/2

                elif vj[i] < vk[i] < wk[i] < wj[i]:
                    if (wj[i] - vk[i]) > (wk[i] - vj[i]):
                        vj[i] = wk[i]

                    else:
                        wj[i] = vk[i]

                elif vk[i] < vj[i] < wj[i] < wk[i]:
                    if (wk[i] - vj[i]) > (wj[i] - vk[i]):
                        vk[i] = wj[i]

                    else:
                        wk[i] = vj[i]

                self.hyperboxes[test_box] = np.array([vk, wk])
                self.hyperboxes[index] = np.array([vj, wj])
                contracted = True
                
        return contracted
    
    def train_pattern(self, X, Y):
        # funkcija koja trenira klasifikator
        target = Y
        
        # ako nemamo tu klasu u klasama
        if target not in self.classes:
            # pravimo hiperboks
            if self.hyperboxes is not None:
                self.hyperboxes = np.vstack((self.hyperboxes, np.array([[X, X]])))
                #print('dodao sam hiperboks: ', self.hyperboxes)
                self.classes = np.hstack((self.classes, np.array([target])))
                #print('dodao sam klasu: ', self.classes)

            else:
                self.hyperboxes = np.array([[X, X]])
                self.classes = np.array([target])
        else:

            # sortiramo pripadnosti svim hiperboksevima za trazenu klasu
            memberships = self.membership(X)
            #print('memberships1: ', memberships)
            memberships[np.where(self.classes != target)] = 0
            #print('memberships2: ', memberships)
            memberships = sorted(list(enumerate(memberships)), key=lambda x: x[1], reverse=True)
            #print('memberships3: ', memberships)
            
            # Expand the most suitable hyperbox
            count = 0
            while True:
                index = memberships[count][0]
                min_new = np.minimum(self.hyperboxes[index, 0, :], X)
                max_new = np.maximum(self.hyperboxes[index, 1, :], X)
                
                if self.exp_bound * len(np.unique(self.classes)) >= np.sum(max_new - min_new):
                    self.hyperboxes[index, 0] = min_new
                    self.hyperboxes[index, 1] = max_new
                    break
                else:
                    count += 1

                if count == len(memberships):
                    self.hyperboxes = np.vstack((self.hyperboxes, np.array([[X, X]])))
                    self.classes = np.hstack((self.classes, np.array([target])))
                    index = len(self.hyperboxes) - 1
                    break
                    
            contracted = self.overlap_contract(index)
        
    def fit(self, X, Y):
        '''
        Wrapper for train_pattern
        '''
        for x, y in zip(X, Y):
            self.train_pattern(x, y)

    
    
    def predict(self, X, k):
        min_pts = self.hyperboxes[:, 0, :]
        max_pts = self.hyperboxes[:, 1, :]
      
        n_classes = len(np.unique(self.classes))
        cl = np.zeros(n_classes)
        
        distance = []
        for i in range(len(min_pts)):
            x1 = min_pts[i][0]
            y1 = min_pts[i][1]
            x2 = max_pts[i][0]
            y2 = max_pts[i][1]
            
            if(x1 == x2 and y1 == y2):
                d = abs(math.sqrt((x1-X[0])**2 + (y1 - X[1])**2))
                distance.append(d)
            elif(x1 == x2):
                d = min(abs(math.sqrt((x1-X[0])**2 + (y1 - X[1])**2)), 
                       abs(math.sqrt((x2-X[0])**2 + (y2 - X[1])**2)))
                distance.append(d)
            elif(y1 == y2):
                d = min(abs(math.sqrt((x1-X[0])**2 + (y1 - X[1])**2)), 
                       abs(math.sqrt((x2-X[0])**2 + (y2 - X[1])**2)))
                distance.append(d)
            else:
                #a = y2-y1
                #b = x2-x1
                #c = (y2-y1)*x1 + (x2-x1)*y1

                #print('iteration: ', i, end='\n')
                #print('a, b, c: ', a, " ", b, " ", c, end='\n')

                #d = abs(a*X[0] + b*X[1] + c) / math.sqrt(a*a + b*b)
                print('(x1, y1): ', x1, " ", y1, end="\n")
                print('(x2, y2): ', x2, " ", y2, end="\n")
                d = abs((y2-y1)*X[0] - (x2-x1)*X[1] + x2*y1 - y2*x1) / math.sqrt((y2-y2)**2 + (x2-x1)**2)
                distance.append(d)

        distance = sorted(list(enumerate(distance)), key=lambda x: x[1])
        print('distance: ', distance, end='\n')
        
        distance = distance[:k]
        print('distance_k: ', distance, end='\n')
        
        distance_index = []
        for i in range(len(distance)):
            distance_index.append(distance[i][0])
        '''
        print('distance_index: ', distance_index)
        print('classes: ', self.classes)
        print('unique cl: ', np.unique(self.classes))
        print('n_uni: ', n_classes)
        '''
    
        for i in range(len(distance_index)):
            index = distance_index[i]
            _class = self.classes[index]
            cl[_class] += 1
            print('cl: ', cl, end = "\n")
        
        final_class = np.argmax(cl)
        
        return final_class
        
    def score(self, X, Y, k):
        '''
        Scores the classifier
        '''
        count = 0
        for x, y in zip(X, Y):
            #self.predict(x, k)
            pred = self.predict(x, k)
            print('predict', pred)
            print('y: ', y)
            if y == pred:
                count += 1
        print(count)
        print(len(Y))
        
        return count / len(Y)

In [3]:
patterns = np.array([[0.1, 0.1],
                     [0.6, 0.6],
                     [0.5, 0.5],
                     [0.4, 0.3]])

classes = np.array([0, 1, 0, 1])

df = pd.read_csv('iris.data', header=None, names=['sepal length', 'sepal width', 'petal length', 'petal width', 'class'])

df = df[~(df['class']=='Iris-virginica')]

#df.head()

df.replace(to_replace='Iris-setosa', value=0, inplace=True)

df.replace(to_replace='Iris-versicolor', value=1, inplace=True)

print(df.sample(frac=1))

X_train = df[['sepal length', 'petal length']].values
Y_train = df['class'].values

_max = np.max(X_train, axis=0)
_min = np.min(X_train, axis=0)
X_train = (X_train - _min) / (_max - _min)
print(X_train)

    sepal length  sepal width  petal length  petal width  class
85           6.0          3.4           4.5          1.6      1
4            5.0          3.6           1.4          0.2      0
20           5.4          3.4           1.7          0.2      0
67           5.8          2.7           4.1          1.0      1
48           5.3          3.7           1.5          0.2      0
..           ...          ...           ...          ...    ...
56           6.3          3.3           4.7          1.6      1
88           5.6          3.0           4.1          1.3      1
86           6.7          3.1           4.7          1.5      1
70           5.9          3.2           4.8          1.8      1
41           4.5          2.3           1.3          0.3      0

[100 rows x 5 columns]
[[0.2962963  0.09756098]
 [0.22222222 0.09756098]
 [0.14814815 0.07317073]
 [0.11111111 0.12195122]
 [0.25925926 0.09756098]
 [0.40740741 0.17073171]
 [0.11111111 0.09756098]
 [0.25925926 0.12195122]
 [0.0370

In [4]:
X_test, Y_test = X_train[-20:], Y_train[-20:]
X_train, Y_train = X_train[:-20], Y_train[:-20]
(Y_test)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [5]:
clf2 = FuzzyMMC(sensitivity=1, exp_bound=0.1)

In [6]:
clf2.fit(X_train, Y_train)

In [7]:
clf2.score(X_test, Y_test, 5)

(x1, y1):  0.2222222222222224   0.09756097560975609
(x2, y2):  0.2962962962962962   0.12195121951219513
(x1, y1):  0.11111111111111104   0.0731707317073171
(x2, y2):  0.14814814814814828   0.12195121951219513
(x1, y1):  0.03703703703703723   0.0731707317073171
(x2, y2):  0.07407407407407414   0.09756097560975609
(x1, y1):  0.14814814814814828   0.09756097560975609
(x2, y2):  0.18518518518518517   0.1463414634146342
(x1, y1):  0.2592592592592593   0.1463414634146342
(x2, y2):  0.2962962962962962   0.17073170731707318
(x1, y1):  0.3333333333333334   0.09756097560975609
(x2, y2):  0.37037037037037035   0.12195121951219513
(x1, y1):  0.888888888888889   0.902439024390244
(x2, y2):  1.0   0.9756097560975611
(x1, y1):  0.7037037037037038   0.853658536585366
(x2, y2):  0.8518518518518517   0.902439024390244
(x1, y1):  0.4444444444444445   0.7317073170731708
(x2, y2):  0.5185185185185186   0.853658536585366
(x1, y1):  0.2222222222222224   0.5609756097560976
(x2, y2):  0.2592592592592593   0.60

cl:  [0. 2.]
cl:  [0. 3.]
cl:  [0. 4.]
cl:  [0. 5.]
predict 1
y:  1
(x1, y1):  0.2222222222222224   0.09756097560975609
(x2, y2):  0.2962962962962962   0.12195121951219513
(x1, y1):  0.11111111111111104   0.0731707317073171
(x2, y2):  0.14814814814814828   0.12195121951219513
(x1, y1):  0.03703703703703723   0.0731707317073171
(x2, y2):  0.07407407407407414   0.09756097560975609
(x1, y1):  0.14814814814814828   0.09756097560975609
(x2, y2):  0.18518518518518517   0.1463414634146342
(x1, y1):  0.2592592592592593   0.1463414634146342
(x2, y2):  0.2962962962962962   0.17073170731707318
(x1, y1):  0.3333333333333334   0.09756097560975609
(x2, y2):  0.37037037037037035   0.12195121951219513
(x1, y1):  0.888888888888889   0.902439024390244
(x2, y2):  1.0   0.9756097560975611
(x1, y1):  0.7037037037037038   0.853658536585366
(x2, y2):  0.8518518518518517   0.902439024390244
(x1, y1):  0.4444444444444445   0.7317073170731708
(x2, y2):  0.5185185185185186   0.853658536585366
(x1, y1):  0.222222

(x1, y1):  0.7037037037037038   0.853658536585366
(x2, y2):  0.8518518518518517   0.902439024390244
(x1, y1):  0.4444444444444445   0.7317073170731708
(x2, y2):  0.5185185185185186   0.853658536585366
(x1, y1):  0.2222222222222224   0.5609756097560976
(x2, y2):  0.2592592592592593   0.6097560975609757
(x1, y1):  0.5555555555555555   0.7317073170731708
(x2, y2):  0.6666666666666665   0.7804878048780489
(x1, y1):  0.5925925925925928   0.902439024390244
(x2, y2):  0.7407407407407407   0.9512195121951221
(x1, y1):  0.4814814814814814   0.6097560975609757
(x2, y2):  0.5185185185185186   0.7073170731707318
(x1, y1):  0.7777777777777779   0.8048780487804879
(x2, y2):  0.888888888888889   0.829268292682927
distance:  [(16, 0.03658536585365804), (24, 0.044346658387113005), (21, 0.04878048780487814), (1, 0.07317073170732151), (4, 0.09756097560975321), (20, 0.09756097560975618), (23, 0.09756097560975668), (15, 0.12195121951219585), (22, 0.21951219512194844), (17, 0.21951219512195078), (18, 0.2682

(x2, y2):  0.6666666666666665   0.7804878048780489
(x1, y1):  0.5925925925925928   0.902439024390244
(x2, y2):  0.7407407407407407   0.9512195121951221
(x1, y1):  0.4814814814814814   0.6097560975609757
(x2, y2):  0.5185185185185186   0.7073170731707318
(x1, y1):  0.7777777777777779   0.8048780487804879
(x2, y2):  0.888888888888889   0.829268292682927
distance:  [(16, 0.012195121951219348), (23, 0.03252032520325256), (20, 0.06504065040650463), (17, 0.07317073170731742), (22, 0.07317073170731865), (21, 0.09756097560975553), (15, 0.12195121951219585), (24, 0.1330399751613395), (18, 0.170731707317075), (1, 0.17073170731707615), (4, 0.1951219512195101), (19, 0.19911682196804847), (3, 0.39024390243902396), (9, 0.46341463414634115), (11, 0.5609756097560972), (0, 0.5853658536585363), (14, 0.6033873969437946), (7, 0.6097560975609757), (2, 0.6197968840879533), (10, 0.652537160514508), (12, 0.6869323275144178), (6, 0.732644074548413), (13, 0.7533344579338384), (8, 0.8804214951793151), (5, 0.9168

(x1, y1):  0.7777777777777779   0.8048780487804879
(x2, y2):  0.888888888888889   0.829268292682927
distance:  [(23, 0.00813008130081339), (16, 0.03658536585365879), (20, 0.04065040650406545), (22, 0.0487804878048786), (15, 0.09756097560975568), (17, 0.09756097560975606), (21, 0.12195121951219498), (1, 0.146341463414637), (24, 0.1478642045062736), (4, 0.17073170731707096), (19, 0.19150219007247196), (18, 0.19512195121951376), (3, 0.36585365853658475), (9, 0.43902439024390205), (11, 0.5365853658536581), (0, 0.5609756097560973), (14, 0.5807810008064033), (7, 0.5853658536585366), (2, 0.5958178091008466), (10, 0.6316921449245794), (12, 0.6626895221106561), (6, 0.7082860891696893), (13, 0.7304823185034006), (8, 0.8588738657822179), (5, 0.8968037978872843)]
distance_k:  [(23, 0.00813008130081339), (16, 0.03658536585365879), (20, 0.04065040650406545), (22, 0.0487804878048786), (15, 0.09756097560975568)]
cl:  [0. 1.]
cl:  [0. 2.]
cl:  [0. 3.]
cl:  [0. 4.]
cl:  [0. 5.]
predict 1
y:  1
20
20


1.0